In [7]:
#1 读取Excel文件

import pandas
import math
import scipy.optimize as sco
import numpy as np

filename = "原始.xlsx"
path = "D:\\熵权\\"
data = pandas.read_excel(path+filename,sheet_name=0)
#保留原始数据
data_yuanshi = data.copy()
print(data)


                 漏斗  地灾       经济损         高程         降水         温差    人数  \
0     181015.865792   0  357436.0  30.113244  21.390701  33.582880     0   
1      64755.509011   0  357436.0  24.416763  22.191440  33.702040  1232   
2      17572.321407   0  357436.0  26.415722  21.741275  34.076674     0   
3          0.000000   0  186154.0  45.854378  22.654401  33.264549   343   
4       7364.964501   0  357436.0  49.383379  22.140590  33.415480  1671   
...             ...  ..       ...        ...        ...        ...   ...   
1090    2352.078201   0  186154.0  41.212604  23.140989  32.302056  2399   
1091       0.000000   2  186154.0  57.027415  23.377501  32.335350     0   
1092   89552.556152   0  186154.0  54.438883  23.567540  32.338740  1000   
1093   35353.016393   0  186154.0  44.420131  22.932312  32.521276  1400   
1094       0.000000   0  186154.0  55.781662  23.166150  32.222549   174   

                 房屋   四地类乘以权重  
0       4439.310059  1.409041  
1      85926.383561  2.

In [8]:
#2 归一化化数据

max = data.max()
min = data.min()
cols = data.columns
#归一化数据
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        data.iloc[i,j] = (float(data.iloc[i,j])-float(min[cols[j]]))/(float(max[cols[j]])-float(min[cols[j]]))
data_guiyi = data.copy()
#转数组
X = data_guiyi.to_numpy()
print(X)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_10580\1835461489.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.04353356890459364' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.iloc[i,j] = (float(data.iloc[i,j])-float(min[cols[j]]))/(float(max[cols[j]])-float(min[cols[j]]))
C:\Users\Administrator\AppData\Local\Temp\ipykernel_10580\1835461489.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.3333333333333333' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.iloc[i,j] = (float(data.iloc[i,j])-float(min[cols[j]]))/(float(max[cols[j]])-float(min[cols[j]]))


[[0.09444039 0.         0.74513423 ... 0.         0.00906997 0.22012211]
 [0.03378453 0.         0.74513423 ... 0.04353357 0.17555656 0.31643773]
 [0.00916791 0.         0.74513423 ... 0.         0.05427544 0.23186357]
 ...
 [0.04672175 0.         0.38234621 ... 0.03533569 0.1071177  0.39062321]
 [0.01844453 0.         0.38234621 ... 0.04946996 0.18327785 0.40085547]
 [0.         0.         0.38234621 ... 0.00614841 0.0291378  0.10034997]]


In [9]:
# 3.输入变量的边界条件
bnds = ((0.2625,0.356),(0.14925,0.326),(0.023,0.09525),
        (0.026,0.093),(0.015,0.0795),(0.034,0.0705),
        (0.1,0.11025),(0.071,0.081),(0.039,0.0685))


# 2.输入约束条件，字典形式传入
cons = ({'type':'eq','fun':lambda w:np.sum(w)-1})


In [10]:
# 1.写出优化函数
def f(w):
    w = np.array(w)
    # 计算每列的均值
    column_means = np.mean(X, axis=0)
    
    # 计算加权方差
    weighted_variances = np.zeros(len(w))
    for i, x_col in enumerate(X.T):
        weighted_variances[i] = w[i] * ((x_col - column_means[i]) ** 2).sum()

    result = weighted_variances.sum()
    return -result
 



    
    # 计算加权方差
    #weighted_variance = np.average((X * w).sum(axis=0) / w.sum() - (X ** 2 * w).sum(axis=0) / w.sum() ** 2, axis=1)
    #return weighted_variance

In [11]:
# 优化：
result = sco.minimize(f,[0.2625,0.14925,0.023,0.026,0.015,0.034,0.1,0.071,0.039],method='SLSQP',bounds=bnds,constraints=cons)
print(result)
print("==========================")


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -31.618659796276447
       x: [ 2.625e-01  1.893e-01  9.525e-02  9.300e-02  7.950e-02
            7.050e-02  1.000e-01  7.100e-02  3.900e-02]
     nit: 2
     jac: [-1.744e+00 -2.270e+01 -9.251e+01 -5.528e+01 -4.779e+01
           -9.917e+01 -1.844e+00 -1.519e+01 -2.203e+01]
    nfev: 20
    njev: 2
